# EDA and Data Cleaning

In [1]:
# Import the pandas library to be used in the task
import pandas as pd

# Ignore warnings - warning were used in the development of the notebook, and removed in the final version
import warnings
# warnings.filterwarnings('ignore')

## Loading and viewing of the data

In [2]:
# Load the dataset to be analysed
unfinished_housing_df = pd.read_csv('unfinished_housing_survey_2017.csv', encoding= 'unicode_escape')
unfinished_housing_df.info()
unfinished_housing_df.shape

FileNotFoundError: [Errno 2] No such file or directory: 'unfinished_housing_survey_2017.csv'

In [ ]:
# Initial viewing of the dataframe
unfinished_housing_df.head(10)

In [ ]:
# Initial viewing of the dataframe
unfinished_housing_df.tail(10)

In [ ]:
# Check that the Department Ref Number is a unique identifier

if unfinished_housing_drop_df['Department Ref Number'].is_unique:
    print("Department Ref Number is a unique identifier")
    # if true, change to type = 'object' as it is not numerical data
    unfinished_housing_drop_df['Department Ref Number'] = unfinished_housing_drop_df['Department Ref Number'].astype('object')
else:
    print("Department Ref Number is not a unique identifier")
    


### Dealing with null values

In [ ]:
# Scan the dataframe for null values
unfinished_housing_df.isnull().sum()

In [ ]:
# Drop unnamed column, and any rows with entirely null data

unfinished_housing_drop_df = unfinished_housing_df.drop(unfinished_housing_df.columns[55], axis=1)
unfinished_housing_drop_df.dropna(axis=0, inplace=True, how='all')

# Drop any rows that have a null Department Ref Number - the unique identifier that will be used 

unfinished_housing_drop_df.dropna(subset = ['Department Ref Number'], inplace=True)

# Display any remaining columns that have null values

null_columns = unfinished_housing_drop_df.isnull().sum()
null_columns = null_columns[null_columns > 0]
print(null_columns)


In [ ]:
# Test for any row that has a large number of null values

null_row_test = unfinished_housing_drop_df.isnull().sum(axis=1)
num_rows_returned = unfinished_housing_drop_df.shape[0]

i = 1
while num_rows_returned > 1:
    # This while loop tests for how many rows have a single null value, then two null values, and continues 
    # until it isolates and prints the row with the most null values
    row_test = unfinished_housing_drop_df[null_row_test == i]
    num_rows_returned = row_test.shape[0]
    i += 1

print(unfinished_housing_drop_df[null_row_test > i])


In [ ]:
# Check whether there is a large number of null values in this row

null_count = unfinished_housing_drop_df.iloc[291].isnull().sum()
null_count

In [ ]:
# Drop the row that has significantly less data than will be useful, and check remaining null columns

unfinished_housing_drop2_df = unfinished_housing_drop_df.drop([291])

null_columns = unfinished_housing_drop2_df.isnull().sum()
null_columns = null_columns[null_columns > 0]
print(null_columns)

## Ensuring all data is for the correct timeframe

In [ ]:
# Confirm that all sites were surveyed in 2017, to make sure no data from previous years was included
unfinished_housing_drop2_df['Survey Date'].unique()


## Cleaning the data

In [ ]:
# Check for Syntax in columns with expected Yes/No values

print(unfinished_housing_drop2_df['Construction Activity (Y/N)'].unique())
print(unfinished_housing_drop2_df['Building Site Only'].unique())

In [ ]:
# Create a loop to scan these columns to standardise the Yes/No strings, as they will be used in the Machine Learning section

# Assign column names
Yes_No_columns = ['Construction Activity (Y/N)','Building Site Only']

# Scan each column for strings containing commas, remove the commas, and convert to integer values

for col in Yes_No_columns:
    # This loop scans the columns in question, and standardises the objects    
    unfinished_housing_drop2_df[col] = unfinished_housing_drop2_df[col].str.replace('yes','Yes')
    unfinished_housing_drop2_df[col] = unfinished_housing_drop2_df[col].str.replace('Yes\n','Yes')
    unfinished_housing_drop2_df[col] = unfinished_housing_drop2_df[col].str.replace('No ','No')
    unfinished_housing_drop2_df[col] = unfinished_housing_drop2_df[col].str.replace('no','No')    
    unfinished_housing_drop2_df[col] = unfinished_housing_drop2_df[col].str.replace('No\n','No')

    
print(unfinished_housing_drop2_df['Construction Activity (Y/N)'].unique())
print(unfinished_housing_drop2_df['Building Site Only'].unique())


## Removing columns

Details of the construction processes not to be used in the analysis are now set aside:

In [ ]:
# A new dataframe is created to focus on housing type, construction activity, occupancy and location

unfinished_construction_df = unfinished_housing_drop2_df[['Department Ref Number', 'Construction Activity (Y/N)', 'County',
                                                          'Detached Units', 'Semi-detached Units', 'Terraced Units', 
                                                          'Apartment Units', 'Duplex Units', 'Total All Units', 'Occupied Houses',
                                                          'Vacant Houses','No Start Houses', 'Occupied Apartments', 
                                                          'Vacant Apartments','No Start Apartments', 'Building Site Only']]

unfinished_construction_df.info()
unfinished_construction_df.shape

In [ ]:
# Initial viewing of the new dataframe
unfinished_construction_df.head()

In [ ]:
# Initial viewing of the new dataframe
unfinished_construction_df.tail()

In [ ]:
unfinished_construction_df['County'].unique()

## Geographical overview,

To give a geographical overview, council areas are merged into counties. 


In [ ]:
# Rename council areas to their respective counties. 'Kerry ' is also merged with 'Kerry'
unfinished_construction_df['County'] = unfinished_construction_df['County'].replace({'Cork City': 'Cork', 'Cork County': 'Cork', 'Galway Co.': 'Galway',
                                                                                    'Galway City': 'Galway', 'Dublin City': 'Dublin', 'DLR ': 'Dublin',
                                                                                    'DLR': 'Dublin', 'Fingal': 'Dublin', 'Kerry ': 'Kerry'})


## Secondary Dataset

Loading and preparing population data so that per capita analysis may be used


In [ ]:
# Load the second dataset
population_df = pd.read_csv('population-census-2016.csv', encoding= 'unicode_escape')
population_df.info()
population_df.shape

In [ ]:
# Initial viewing of new dataset
population_df.head()

In [ ]:
# Initial viewing of new dataset
population_df.tail()

## Organising the columns

As the titles in the orginal file, coonverted from an Excel spreadsheet, appeared over two columns, the column headings are renamed using the information in the first row. The first column is renamed 'County' to correspond with the original dataframe

In [ ]:
population_df = population_df.rename(columns={"Region and county": "County", "Unnamed: 2": "County Population"})

# The first row can now be dropped

population_df = population_df.drop([0])

# The columns relating to the 2011 data and the column containing only null values are also dropped

population_df = population_df.drop(['Population', 'Unnamed: 3', 'Unnamed: 4','Share of total population', 'Unnamed: 6'], axis = 1)

## Cleaning the data

Identifying and removing noise from the data


In [ ]:
# Remove commas from strings
population_df['County Population'] = population_df['County Population'] .str.replace(',','')

# Ensure the column contains numerical data, not objects
population_df['County Population'] = population_df['County Population'].astype('float')

# multiply the column values by 1000 to represent the total population
population_df['County Population'] = population_df['County Population'] * 1000



## Standardise the geographical areas to correspond with the original dataframe

In [ ]:
population_df['County'].unique()

In [ ]:
population_df.head(40)

In [ ]:
# A new dataframe is created, with the rows renamed (as counties) to correspond with the data in the original data frame.

population_county_df = population_df
population_county_df['County'] = population_df['County'].replace({'Cork City': 'Cork', 'Cork County': 'Cork', 'Galway County': 'Galway',
                                                                                    'Galway City': 'Galway'})

# The data is the grouped by county
population_county_df = population_county_df.groupby('County').sum()

# This population data can now be merged with the unfinished construction dataframe

unfinished_construction_population_df = pd.merge(unfinished_construction_df, population_county_df, how = 'left', on = 'County')

# Create a CSV of this dataframe to be used in subsequent Jupyter notebooks.

unfinished_construction_population_df.to_csv('unfinished_construction_population_df.csv', index = 'False')

The data is now prepared for analysis in the statistics and ML sections